In [1]:
import torch, torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import pandas as pd
from IPython.display import display
from pathlib import Path

"""
Скрипт сравнения распределений P(next_token) с поддержкой:
----------------------------------------------------------------
1. **Внутренних промптов** (список в коде) **или**
2. **Промптов из текстового файла** в той же директории.

Флаг `USE_FILE_PROMPTS` выбирает источник.
Если `True`, файл `PROMPT_FILE` читается целиком и используется как **один** длинный prompt.

Модификация «summary + слова» печатает краткое summary исходного prompt‑а.
Добавлена **безопасная токенизация**: если длиннее лимита модели (1024 для GPT‑2),
текст автоматически усечётся, а в консоль выведется предупреждение.
"""

# ---------- ПАРАМЕТРЫ ----------
MODEL_NAME   = "gpt2"
STEPS        = 3
DEVICE       = "cuda" if torch.cuda.is_available() else "cpu"

# Флаг, управляющий включением суммаризации как отдельной модификации.
# Если PROMPTS короткие или суммаризация не нужна, установите False.
SUMMARY_ENABLED = True  # ← меняйте по необходимости

# Суммаризация
SUMM_MODEL   = "sshleifer/distilbart-cnn-12-6"
CUSTOM_WORDS = "Сводка:"

# Выбор источника промптов
USE_FILE_PROMPTS = True             # ← переключите на True для чтения из файла
PROMPT_FILE      = "prompt.txt"      # UTF‑8 файл в той же папке

# ---------- 1. Модели ----------
_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
_model     = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE).eval()

_summarizer = pipeline(
    "summarization",
    model=SUMM_MODEL,
    tokenizer=SUMM_MODEL,
    device=0 if DEVICE=="cuda" else -1,
)

CTX_LIMIT = _model.config.n_positions  # 1024 для GPT‑2

# ---------- 2. Вспомогательные функции ----------

def _next_token_probs(ids):
    """Вернёт софтмакс‑распределение для следующего токена."""
    with torch.no_grad():
        logits = _model(ids).logits[0, -1]
    return torch.softmax(logits, dim=-1).cpu()


def _kl(p, q):
    return F.kl_div(p.log(), q, reduction="batchmean").item()


def _cos(p, q):
    return F.cosine_similarity(p, q, dim=0).item()


def _multi_step(prompt: str, n: int):
    """Возвращает список распределений для n последующих токенов.
    Если prompt длиннее контекст‑лимита, автоматически усечёт и предупредит."""
    token_ids = _tokenizer.encode(prompt)
    if len(token_ids) > CTX_LIMIT:
        print(f"⚠️ Промпт содержит {len(token_ids)} токенов и будет усечён до {CTX_LIMIT}.")
    ids = _tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=CTX_LIMIT
        )["input_ids"].to(DEVICE)

    dists = []
    for _ in range(n):
        probs = _next_token_probs(ids)
        dists.append(probs)
        next_id = probs.argmax().unsqueeze(0).unsqueeze(0).to(DEVICE)
        ids = torch.cat([ids, next_id], dim=1)
    return dists

# ---------- 3. Источник промптов ----------
if USE_FILE_PROMPTS:
    path = Path(PROMPT_FILE)
    if not path.is_file():
        raise FileNotFoundError(f"Файл {PROMPT_FILE} не найден")
    PROMPTS = [path.read_text(encoding="utf-8").strip()]
else:
    PROMPTS = [
        "Why the stock market is expected to",
        "The future of artificial intelligence depends on",
    ]
    

# ---------- 4. Модификации ----------
def add_typo_first_e(p: str) -> str:
    """Replace the first occurrence of 'e' with '3'."""
    return p.replace("e", "3", 1)

def add_salutation(p: str) -> str:
    """Prepend a greeting line."""
    return "Dear GPT,\n" + p


MODS = {
    "префикс ======":   lambda p: "="*10 + p,
     "префикс question(англ)":  lambda p: "I have a question. " + p,
    "префикс вопрос":  lambda p: "У меня есть вопрос. " + p,
    "суффикс 10 лет?": lambda p: p + " in the next decade?",
}

# --- Модификация с суммаризацией и печатью ---

def summary_mod(custom: str = CUSTOM_WORDS):
    """Возвращает модификатор, который:
    ▸ один раз печатает summary для каждого уникального промпта (до расчётов),
    ▸ кэширует результат, чтобы в блоке 5 печать не дублировалась,
    ▸ добавляет summary в начало текста.
    """
    cache = {}

    def _fn(p: str):
        if p not in cache:  # печатаем только при первом появлении промпта
            summary = _summarizer(p, max_length=60, min_length=15, do_sample=False)[0]["summary_text"]
            print(f"\n[SUMMARY]\n{summary}\n")
            cache[p] = summary
        else:
            summary = cache[p]
        return f"{custom} {summary}\n\n{p}"

    return _fn

# создаём функцию‑модификатор и сразу кладём её в словарь
if SUMMARY_ENABLED:
    summary_fn = summary_mod()
    MODS["summary + слова (print)"] = summary_fn

 # ---------- 4.1. Предварительный вывод Summary (до расчётов) ----------
    for _p in PROMPTS:
        summary_fn(_p)
# пройдёмся по всем промптам и вызовем summary_fn, чтобы вывести summary ЗАРАНЕЕ

Device set to use cuda:0



[SUMMARY]
 Python’s __call__ method will take a list of invoices as input and either build the purchase table or update it . We’ll create an object that represents a Pandas table of a day's product purchases .



In [4]:
PROMPTS

['Let’s come up with an example that uses __call__ in Python.\nWe’ll create an object that represents a Pandas table of a day’s product purchases, with the following columns: product name, unit price, quantity, and the total amount spent on that product.\nThe object’s __call__ method will take a list of invoices as input and either build the purchase table or update it.']

In [5]:
STEPS

3

In [6]:
MODS.items()

dict_items([('префикс ======', <function <lambda> at 0x000002C892FB2AC0>), ('префикс question(англ)', <function <lambda> at 0x000002C8EA4762A0>), ('префикс вопрос', <function <lambda> at 0x000002C8EA475580>), ('суффикс 10 лет?', <function <lambda> at 0x000002C8EA4763E0>), ('summary + слова (print)', <function summary_mod.<locals>._fn at 0x000002C8EA476700>)])

In [7]:
MODS

{'префикс ======': <function __main__.<lambda>(p)>,
 'префикс question(англ)': <function __main__.<lambda>(p)>,
 'префикс вопрос': <function __main__.<lambda>(p)>,
 'суффикс 10 лет?': <function __main__.<lambda>(p)>,
 'summary + слова (print)': <function __main__.summary_mod.<locals>._fn(p: str)>}

In [9]:
# ---------- 5. Расчёт ----------
records = []
for prompt in PROMPTS:
    base = _multi_step(prompt, STEPS)
    for mod_name, mod_fn in MODS.items():
        mod  = _multi_step(mod_fn(prompt), STEPS)
        print()
        print(mod_fn(prompt))
        for step in range(STEPS):
            records.append({
                "prompt": prompt[:80] + ("…" if len(prompt) > 80 else ""),
                "mod":    mod_name,
                "step":   step + 1,
                "cos":    _cos(base[step], mod[step]),
                "kl":     _kl(base[step],  mod[step]),
            })

_df = pd.DataFrame(records)
pd.set_option("display.precision", 4)


==========Let’s come up with an example that uses __call__ in Python.
We’ll create an object that represents a Pandas table of a day’s product purchases, with the following columns: product name, unit price, quantity, and the total amount spent on that product.
The object’s __call__ method will take a list of invoices as input and either build the purchase table or update it.

I have a question. Let’s come up with an example that uses __call__ in Python.
We’ll create an object that represents a Pandas table of a day’s product purchases, with the following columns: product name, unit price, quantity, and the total amount spent on that product.
The object’s __call__ method will take a list of invoices as input and either build the purchase table or update it.

У меня есть вопрос. Let’s come up with an example that uses __call__ in Python.
We’ll create an object that represents a Pandas table of a day’s product purchases, with the following columns: product name, unit price, quantity, an

In [3]:
# ---------- 6. Вывод ----------  
    
# ---------- 6.1 Вывод Cosine----------
for s in range(1, STEPS + 1):
    df = _df[_df.step == s]
    print(f"\n=== Шаг {s} — Cosine ===")
    display(df.pivot(index="prompt", columns="mod", values="cos"))
    #print(f"\n=== Шаг {s} — KL ===")
    #display(df.pivot(index="prompt", columns="mod", values="kl"))
    
# ---------- 6.2 Вывод KL----------
for s in range(1, STEPS + 1):
    df = _df[_df.step == s]
    #print(f"\n=== Шаг {s} — Cosine ===")
    #display(df.pivot(index="prompt", columns="mod", values="cos"))
    print(f"\n=== Шаг {s} — KL ===")
    display(df.pivot(index="prompt", columns="mod", values="kl"))

# ---------- 7. Лимит строк (без ошибки в Jupyter) ----------
import inspect, sys
try:
    if len(inspect.getsource(sys.modules[__name__]).splitlines()) > 175:
        print("⚠️ Файл приближается к лимиту, рассмотрите рефакторинг.")
except (TypeError, OSError):
    pass


=== Шаг 1 — Cosine ===


mod,summary + слова (print),префикс ======,префикс question(англ),префикс вопрос,суффикс 10 лет?
prompt,,,,,
Let’s come up with an example that uses __call__ in Python.\nWe’ll create an obje…,0.4758,0.9996,0.9994,0.9995,0.8843



=== Шаг 2 — Cosine ===


mod,summary + слова (print),префикс ======,префикс question(англ),префикс вопрос,суффикс 10 лет?
prompt,,,,,
Let’s come up with an example that uses __call__ in Python.\nWe’ll create an obje…,0.0109,0.992,0.9937,0.9967,0.9417



=== Шаг 3 — Cosine ===


...

mod,summary + слова (print),префикс ======,префикс question(англ),префикс вопрос,суффикс 10 лет?
prompt,,,,,
Let’s come up with an example that uses __call__ in Python.\nWe’ll create an obje…,2.1358e-05,1.2853e-07,2.3676e-07,1.6544e-07,4.0095e-05



=== Шаг 2 — KL ===


mod,summary + слова (print),префикс ======,префикс question(англ),префикс вопрос,суффикс 10 лет?
prompt,,,,,
Let’s come up with an example that uses __call__ in Python.\nWe’ll create an obje…,0.0001,5.9675e-07,5.4638e-07,3.0618e-07,3.7681e-06



=== Шаг 3 — KL ===


mod,summary + слова (print),префикс ======,префикс question(англ),префикс вопрос,суффикс 10 лет?
prompt,,,,,
Let’s come up with an example that uses __call__ in Python.\nWe’ll create an obje…,0.0004,1.3054e-07,2.7301e-07,2.0257e-07,5.7380e-06
